In [6]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import random
from collections import defaultdict
import pandas as pd

import torch.nn.functional as F



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_indexes(arr, value):
    indexes = []
    for i in range(len(arr)):
        if arr[i] == value:
            indexes.append(i)
    return indexes

def get_length_per_class(dataloader, classes):
    class_counts = defaultdict(int)
    total = 0
    for batch in dataloader:
        _, labels = batch 
        labels = labels.numpy().tolist()
        for label in labels:
            class_counts[label] += 1
            total +=1

    class_counts = dict(sorted(class_counts.items()))
    for class_label, count in class_counts.items():
        print(f"Class {classes[class_label]}: {count} samples out of {total}")
def load_data(data_dir,
                           batch_size,
                           data_type,
                           noise_type,
                           noise_percentage,                           
                           transform,                           
                           data_percentage=1,
                           show_classes = False, random_seed=21):
    
    if noise_type == "None":
        noise_type = ""
        noise_percentage = ""
    else:
        noise_type = "/" + str(noise_type)
        noise_percentage = "/" + str(noise_percentage)
    path = data_dir + noise_type + "/" + data_type + noise_percentage
    print("path: ", path)
    dataset = ImageFolder(root=path, transform=transform)
    original_classes = dataset.classes 
    num_samples = len(dataset)
    indices = list(range(num_samples))

    labels = dataset.targets
    class_to_idx = dataset.class_to_idx
    needed_length = int(num_samples*data_percentage/100)
    expected_length_per_class = int(needed_length/len(original_classes))
    print(f"needed_length: {needed_length}, expected_length_per_class: {expected_length_per_class}")
    if data_percentage != 100:
        new_indices = []
        for key, value in class_to_idx.items():
            all_indixes_of_class = get_indexes(labels, value)
            new_indices.extend(all_indixes_of_class[:expected_length_per_class])
    else:
        new_indices = indices
    length_dataset = len(new_indices)
    print("length of final dataset:", length_dataset)

    
    # sampler = SubsetRandomSampler(new_indices)

    dataloader = DataLoader(dataset, sampler=new_indices, batch_size=batch_size)

    if show_classes:
        get_length_per_class(dataloader, original_classes)
        
    random.shuffle(new_indices)

   
    dataloader = DataLoader(dataset, sampler=new_indices, batch_size=batch_size)

    return dataloader, length_dataset, original_classes


def train_model(model, criterion, optimizer, scheduler, num_epochs=25, batch_show = 1792):
    since = time.time()
    valid_acc = []
    train_acc = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0
            l = 0

            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.clone().detach().to(device)
                labels = labels.clone().detach().to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    if not isinstance(outputs, torch.Tensor):
                        outputs = outputs.logits
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                l += len(inputs)
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'train':
                scheduler.step()
                train_acc.append(epoch_acc.item())
            else:
                valid_acc.append(epoch_acc.item())
                
            
            
            
            print('\n{} Loss: {:.4f} Acc: {:.4f}\n'.format(
                phase, epoch_loss, epoch_acc))

            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    
   

    accuracy_history = [train_acc, valid_acc]
    model.load_state_dict(best_model_wts)
    return model, best_acc.item(), accuracy_history
    
    





print('ready')

ready


In [7]:
transform = transforms.Compose([
    transforms.Resize((227, 227)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4680, 0.4647, 0.3441], std=[0.2322, 0.2272, 0.2394]) 
])   

noise_type = "gaussian_noise"
noise_percentage = 10
data_percentage = 100
total_size = 21000

train_size = data_percentage*total_size/100
data_dir = '/kaggle/input/vegetableimages/vegetable_images'

train_loader, train_size, classes = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "train",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader, valid_size, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader_with_noise, _, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = noise_type,
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataloaders = {'train':  train_loader, 
               'val': valid_loader_with_noise
               }
dataloaders_with_noise = {'train':  train_loader, 
               'val': valid_loader_with_noise
               }


test_loader,test_size_, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "gaussian_noise",
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)


test_loader_without_noise, _, _ = load_data(data_dir =data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataset_sizes = {'train':  train_size, 
        'val': valid_size,
        'test': test_size_
       }




path:  /kaggle/input/vegetableimages/vegetable_images/train
needed_length: 15000, expected_length_per_class: 1000
length of final dataset: 15000
path:  /kaggle/input/vegetableimages/vegetable_images/validation
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/gaussian_noise/validation/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/gaussian_noise/test/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/test
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000


In [8]:
dataset_sizes

{'train': 15000, 'val': 3000, 'test': 3000}

In [9]:
def load_alexnet():
    alexnet = models.alexnet(weights="DEFAULT")
    in_features = alexnet._modules['classifier'][-1].in_features
    out_features = len(classes)
    alexnet._modules['classifier'][-1] = nn.Linear(in_features, out_features, bias=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    alexnet = alexnet.to(device)
    for param in alexnet.parameters():
        param.requires_grad = True
    return alexnet
    
    
    

# Hyperparameter tuning 

In [12]:
criterion = nn.CrossEntropyLoss()
learning_rates = [0.005]
momentums = [0, 0.1]
steps = [3, 4]
gammas = [0.1,  0.5]
c = 0
columns = ['model', 'num_epochs', 'learning_rate', 'momentum', 'step', 'gamma', 'accuracy', 'history']
num_epochs = 10
ht_results = pd.DataFrame(columns = columns)
best_valid_score = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for lr in learning_rates:
    for m in momentums:
        for s in steps:
            for g in gammas: 
                c+=1
                print(f"STARTING {c} ITERATION")
                alexnet_cr = load_alexnet()
                print(f"Starting to test lr = {lr}, m = {m}, s = {s}, g = {g}")
                optimizer = optim.SGD(alexnet_cr.parameters(), lr= lr, momentum = m)
                step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
                alexnet_cr, valid_score, history = train_model(alexnet_cr, criterion, optimizer, 
                                                      step_lr_scheduler, num_epochs=num_epochs)
                if best_valid_score < valid_score:
                    best_valid_score = valid_score
                    best_alexnet = alexnet_cr
                ht_results.loc[len(ht_results.index)] = ['alexnet', num_epochs, lr, m, s, g, valid_score, history]                
                print(ht_results.loc[len(ht_results.index) - 1])
                
        
        



STARTING 1 ITERATION
Starting to test lr = 0.005, m = 0, s = 3, g = 0.1
Epoch 1/10
----------

train Loss: 0.2006 Acc: 0.9431


val Loss: 0.1068 Acc: 0.9670

Epoch 2/10
----------

train Loss: 0.0280 Acc: 0.9921


val Loss: 0.0758 Acc: 0.9760

Epoch 3/10
----------

train Loss: 0.0156 Acc: 0.9959


val Loss: 0.0478 Acc: 0.9853

Epoch 4/10
----------

train Loss: 0.0120 Acc: 0.9967


val Loss: 0.0433 Acc: 0.9883

Epoch 5/10
----------

train Loss: 0.0069 Acc: 0.9982


val Loss: 0.0536 Acc: 0.9840

Epoch 6/10
----------

train Loss: 0.0048 Acc: 0.9989


val Loss: 0.0492 Acc: 0.9847

Epoch 7/10
----------

train Loss: 0.0037 Acc: 0.9992


val Loss: 0.0456 Acc: 0.9853

Epoch 8/10
----------

train Loss: 0.0038 Acc: 0.9992


val Loss: 0.0434 Acc: 0.9857

Epoch 9/10
----------

train Loss: 0.0033 Acc: 0.9995


val Loss: 0.0453 Acc: 0.9850

Epoch 10/10
----------

train Loss: 0.0032 Acc: 0.9995


val Loss: 0.0501 Acc: 0.9833

Training complete in 15m 41s
Best val Acc: 0.988333
model          

In [13]:
ht_results.to_csv('alexnet_results.csv', index=False)

In [14]:
torch.save(best_alexnet, "best_alexnet.pth")

In [15]:
!zip -r file.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/best_alexnet.pth (deflated 7%)
  adding: kaggle/working/alexnet_results.csv (deflated 83%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)


In [16]:
!ls

alexnet_results.csv  best_alexnet.pth  file.zip


In [17]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip